In [1]:
import requests
from langchain.llms.base import LLM
from typing import Optional, List


class MistralLLM(LLM):
    api_key: str
    model_name: str
    api_url: str = "https://api.mistral.ai/v1/chat/completions"

    @property
    def _llm_type(self) -> str:
        return "mistral"

    def _call(self, system_prompt: str, user_prompt: str,
              stop: Optional[List[str]] = None, max_tokens: int = 150, **kwargs) -> str:

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "max_tokens": max_tokens,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            # "temperature": kwargs.get("temperature", 1),
            # "top_p": kwargs.get("top_p", 1),
            # "stream": kwargs.get("stream", False),
            # "stop": stop or ["string"],
            # "random_seed": kwargs.get("random_seed", 0),
            # "response_format": {"type": "text"},
            # "tools": kwargs.get("tools", []),
            # "tool_choice": kwargs.get("tool_choice", "auto"),
            # "presence_penalty": kwargs.get("presence_penalty", 0),
            # "frequency_penalty": kwargs.get("frequency_penalty", 0),
            # "n": kwargs.get("n", 1),
            # "safe_prompt": kwargs.get("safe_prompt", False)
        }

        response = requests.post(self.api_url, headers=headers, json=payload)
        response_data = response.json()

        if response.status_code != 200:
            raise Exception(f"Mistral API Error: {response_data.get('message', 'Unknown error')}")

        return response_data.get("choices", [{}])[0].get("message", {}).get("content", "")

    def generate(self, system_prompt: str, user_prompt: str, **kwargs) -> str:
        return self._call(system_prompt, user_prompt, **kwargs)

In [2]:
from langchain import LLMChain, PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv('../.env')

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

mistral_llm = MistralLLM(api_key=MISTRAL_API_KEY,
                         model_name="mistral-large-latest")


system_template = "You are an assistant that helps with translations."
user_template = "{input_text}"

system_prompt_template = PromptTemplate(input_variables=[], template=system_template)
user_prompt_template = PromptTemplate(input_variables=["input_text"], template=user_template)

user_input = "Придумай мем про Штирлица и машинное обучение"

formatted_system_prompt = system_prompt_template.format()
formatted_user_prompt = user_prompt_template.format(input_text=user_input)


response = mistral_llm.generate(system_prompt=formatted_system_prompt,
                                user_prompt=formatted_user_prompt,
                                max_tokens=512)
print(response)

Конечно! Вот идея для мема про Штирлица и машинное обучение:

**Изображение:** Штирлиц сидит за столом, окружённый кучей бумаг и книг, на экране компьютера виден график обучения модели.

**Текст:**

**Верхний текст:** "Когда Штирлиц пытается объяснить Мюллеру, почему его модель машинного обучения не может предсказать поведение Гитлера."

**Нижний текст:** "Мюллер: 'Штирлиц, вы уверены, что у вас достаточно данных?'"

**Подпись:** "Штирлиц: 'Данные есть, но Гитлер — это такой аутлаер, что модель просто не справляется.'"

Надеюсь, вам понравится этот мем!


In [1]:
from langchain_mistralai import MistralAIEmbeddings

embed_model = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=input()
        )

/home/danis/miniconda3/envs/p310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/danis/miniconda3/envs/p310/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [10]:
from langchain.vectorstores import Milvus as LC_Milvus

vector_store = LC_Milvus(
            embedding_function=embed_model,
            collection_name='_123',
            connection_args={"uri": "http://localhost:19530"},
            auto_id=True,
        )

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

tokenizer_embed = MistralTokenizer.v1()

def get_token_count_embedding(text: str) -> int:
    return len(tokenizer_embed.instruct_tokenizer.encode_user_content(
        text, is_last=True)[0]) + 2

6

In [22]:
texts = ["Hello, world!", "Привет, мир!", "Hallo, Welt!"] * 700

sum([get_token_count_embedding(text) for text in texts])

14700

In [23]:
await vector_store.aadd_texts(texts)

[454946368562946157,
 454946368562946158,
 454946368562946159,
 454946368562946160,
 454946368562946161,
 454946368562946162,
 454946368562946163,
 454946368562946164,
 454946368562946165,
 454946368562946166,
 454946368562946167,
 454946368562946168,
 454946368562946169,
 454946368562946170,
 454946368562946171,
 454946368562946172,
 454946368562946173,
 454946368562946174,
 454946368562946175,
 454946368562946176,
 454946368562946177,
 454946368562946178,
 454946368562946179,
 454946368562946180,
 454946368562946181,
 454946368562946182,
 454946368562946183,
 454946368562946184,
 454946368562946185,
 454946368562946186,
 454946368562946187,
 454946368562946188,
 454946368562946189,
 454946368562946190,
 454946368562946191,
 454946368562946192,
 454946368562946193,
 454946368562946194,
 454946368562946195,
 454946368562946196,
 454946368562946197,
 454946368562946198,
 454946368562946199,
 454946368562946200,
 454946368562946201,
 454946368562946202,
 454946368562946203,
 454946368562